In [ ]:
!pip install langchain google-search-results langchain-community

In [ ]:
!pip install openai

Search Parameters Adjustment:

Number of Results (num): Increased the number of results fetched to 5. This provides a broader context by fetching more search results.
Filter (filter): Set to "0" to avoid filtering out similar results, allowing more diverse sources to be included.
Sort by Date (sort): Added to prioritize the most recent information, ensuring the returned results are up-to-date.
Combining Snippets:

Snippet Aggregation: Instead of relying on a single top result, the script now aggregates snippets from multiple top search results. This ensures a more comprehensive view of the information available.
Fallback Mechanism:

No Results Handling: Added a fallback message when no relevant snippets are found to handle cases where the search might not return useful information.


In [3]:
import os
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from serpapi import GoogleSearch

# Set up your API keys
serpapi_api_key = 'a300eb2a55918454e2ac5c3a5653467f604586e52aa551e2b6a74be31c9dc08f'
openai_api_key = 'sk-proj-SLD1kkrCM3KhATIGjURlT3BlbkFJk75IwDuvAHX10HbHz8iq'

os.environ["OPENAI_API_KEY"] = openai_api_key

# Initialize the Google Search client
def google_search(query):
    params = {
        "q": query,
        "api_key": serpapi_api_key,
        "num": 5,  # Retrieve top 5 results for more context
        "filter": "0",  # Do not filter similar results to get diverse sources
        "sort": "date"  # Sort by date to get the latest information
    }
    search = GoogleSearch(params)
    results = search.get_dict()
    return results



In [4]:
# Define the prompt template
prompt_template = """
You are a helpful assistant. A user has asked the following question:

{question}

Based on the most recent and reliable information from a Google search, provide a concise and accurate answer.
"""


In [6]:

# Initialize Langchain components
prompt = PromptTemplate(input_variables=["question"], template=prompt_template)
llm = OpenAI(temperature=0.7)


In [7]:

# Create the LLM chain
chain = LLMChain(prompt=prompt, llm=llm)




/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [8]:
# Function to handle the query and response
def get_answer(query):
    search_results = google_search(query)
    snippets = [result['snippet'] for result in search_results.get('organic_results', []) if 'snippet' in result]
    combined_snippets = " ".join(snippets)

    if not combined_snippets:
        combined_snippets = "No relevant information found."

    response = chain.run({"question": combined_snippets})
    return response



In [11]:
# Example usage
if __name__ == "__main__":
    query = input("Enter your question: ")
    answer = get_answer(query)
    print("Answer:", answer)

Enter your question: what is the latests news in AI in china
Answer: 
China plans to establish at least 50 sets of AI standards by 2026, which is expected to significantly impact industrial production and the economy. The country's premier has called for international cooperation in AI, while Baidu, a major AI company, recently announced an upgraded AI model with a user base of 300 million. However, a recent poll shows that both Republican and Democratic voters do not believe that the US needs to rush ahead in AI to beat China.
